In [1]:
# import required libraries
import numpy as np
import pandas as pd

**Import the Dataset**

In [2]:
# Load contraints
country_list = pd.read_excel('D:\Big-Mac-Index-Panel-Data-Analysis\data\constraints.xlsx', sheet_name = 'country-code')
years = pd.read_excel('D:\Big-Mac-Index-Panel-Data-Analysis\data\constraints.xlsx', sheet_name = 'time-period')

# Load datasets
labcomp = pd.read_excel(r'D:\\TimeSeriesProj\\UnCleanedData\\pwt100.xlsx', sheet_name = 2)
ggb = pd.read_csv('D:\Big-Mac-Index-Panel-Data-Analysis\data\DP_LIVE_28052022104927583.csv')

**Labour Compensation Dataset Pre-processing**

In [3]:
# Filter Country List
labcomp['new_countrycode'] = labcomp.apply(lambda x: 'EUZ' if x['currency_unit'] == 'Euro' else x['countrycode'], axis = 1)

labcomp_data_country_filtered = pd.merge(left = country_list, 
                                    right = labcomp, 
                                    how = 'left', 
                                    left_on = 'iso_a3', 
                                    right_on = 'new_countrycode')

keep_cols = ['name', 'countrycode', 'country', 'year', 'labsh', 'new_countrycode']
labcomp_data_country_filtered = labcomp_data_country_filtered[keep_cols]

labcomp_data_country_filtered.head()


,name,countrycode,country,year,labsh,new_countrycode
0,Australia,AUS,Australia,1950,0.680492,AUS
1,Australia,AUS,Australia,1951,0.680492,AUS
2,Australia,AUS,Australia,1952,0.680492,AUS
3,Australia,AUS,Australia,1953,0.680492,AUS
4,Australia,AUS,Australia,1954,0.680492,AUS


In [4]:
eur = labcomp_data_country_filtered[labcomp_data_country_filtered['new_countrycode'] == 'EUZ']
eur = eur.groupby(['year'], as_index=False).mean()
eur['name'] = 'Euro Area'
eur['countrycode'] = 'EUZ'
eur.head()

,year,labsh,name,countrycode
0,1950,0.625031,Euro Area,EUZ
1,1951,0.614484,Euro Area,EUZ
2,1952,0.615965,Euro Area,EUZ
3,1953,0.615387,Euro Area,EUZ
4,1954,0.610143,Euro Area,EUZ


In [5]:
labcomp_data_country_filtered.drop(columns=['countrycode','country'], inplace=True)
labcomp_data_country_filtered = labcomp_data_country_filtered.rename(columns={'new_countrycode':'countrycode'})
labcomp_data_country_filtered = labcomp_data_country_filtered[labcomp_data_country_filtered['countrycode'] != 'EUZ']

In [6]:
labcomp_data_country_filtered = labcomp_data_country_filtered.append(eur)

labcomp_data_country_filtered = labcomp_data_country_filtered[(labcomp_data_country_filtered['year'] >= 2000) & (labcomp_data_country_filtered['year'] <= 2016)]

# drop a few countries 
drop_countries = ['HKG', 'MYS', 'SGP', 'TWN', 'THA']
new_df = labcomp_data_country_filtered[~labcomp_data_country_filtered.countrycode.isin(drop_countries)]
new_df = new_df.rename(columns = {'name':'Country Name', 'year':'Year', 'countrycode': 'Country Code'})

C:\Users\yashv\AppData\Local\Temp\ipykernel_23108\2123043983.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labcomp_data_country_filtered = labcomp_data_country_filtered.append(eur)


In [7]:
path = 'D:\\Big-Mac-Index-Panel-Data-Analysis\\CleanedData\\'
new_df.to_csv(path + "labcomp.csv", index=False)

**Government Balance Dataset Preprocessing**

In [8]:
ggb_data_country_filtered = pd.merge(left = country_list, 
                                    right = ggb, 
                                    how = 'left', 
                                    left_on = 'iso_a3', 
                                    right_on = 'LOCATION')

ggb_data_country_filtered = ggb_data_country_filtered[(ggb_data_country_filtered['TIME'] >= 2000) & (ggb_data_country_filtered['TIME'] <= 2016)]

ggb_data_country_filtered.drop(columns=['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes', 'currency_code', 'iso_a3'], inplace = True)
ggb_data_country_filtered.rename(columns={'name':'Country Name', 'TIME':'Year', "LOCATION": 'Country Code', 'Value': 'GGB'})
ggb_data_country_filtered.head()
## HKG, MYS, SGP, TWN, THA

,name,LOCATION,TIME,Value
0,Australia,AUS,2000.0,-1.292779
1,Australia,AUS,2001.0,-0.743403
2,Australia,AUS,2002.0,0.706782
3,Australia,AUS,2003.0,0.794190
4,Australia,AUS,2004.0,0.927764


In [9]:
ggb_data_country_filtered.to_csv(path + "ggb.csv", index=False)